In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime
import re
from unidecode import unidecode
tqdm.pandas()

In [2]:
page = requests.get("https://thefangirlverdict.com/index/all-reviews/full-list-of-shows/")
soup = BeautifulSoup(page.content)

In [3]:
for p in soup.find_all("p", style="text-align:justify"):
    p.decompose()

In [4]:
names = []
urls = []
grades = []

for x in tqdm(range(700)):
    
    element = soup.select_one("#post-7658 > div > p:nth-child(" + str(x) + ")")
    if element is None:
        continue
    
    kdrama = element.findChildren("a" , recursive=False)
    #print(kdrama)
    grade = element.findChildren("strong", recursive=False)
    #print(grade)
    
    if kdrama:
        names.append(kdrama[0].contents[0])
        urls.append(kdrama[0].get("href"))
        if grade:
            grades.append(grade[0].text)
        else:
            grades.append("NA")

100%|█████████████████████████████████████████| 700/700 [03:27<00:00,  3.38it/s]


In [5]:
df = pd.DataFrame({"source": names, "url": urls, "rating": grades})

In [6]:
# df['rating'] = df['rating'].replace(['A++', 'A+', 'A', 'A-','B++','B+','B','B-','C++', 'C+', 'C', 'C-','D++','D+','D','D-'],['A1', 'A2', 'A3', 'A4','B1','B2','B3','B4','C1', 'C2', 'C3', 'C4','D1','D2','D3','D4'])

In [7]:
df["year"] = df.apply(lambda row: re.search(r'\d+', row["url"]).group(), axis=1)
#df["source"] = df.apply(lambda row: unidecode(row["source"]), axis=1)
df

,source,url,rating,year
0,1% of Anything [Something About 1%],http://thefangirlverdict.com/2017/04/29/flash-...,B+,2017
1,18 Again,https://thefangirlverdict.com/2021/01/08/revie...,A,2021
2,"1st Shop Of Coffee Prince, The [Coffee Prince]",http://thefangirlverdict.com/2019/01/03/review...,A,2019
3,20â€™s [Twenty Years Old],http://thefangirlverdict.com/2015/04/10/flash-...,B+,2015
4,20th Century Boy And Girl,http://thefangirlverdict.com/2017/12/02/flash-...,B,2017
...,...,...,...,...
528,Year In Review: 2016,http://thefangirlverdict.com/2016/12/28/year-i...,NA,2016
529,Year In Review: 2017,http://thefangirlverdict.com/2017/12/31/year-i...,NA,2017
530,Year In Review: 2018,http://thefangirlverdict.com/2018/12/26/year-i...,NA,2018
531,Year In Review: 2019,http://thefangirlverdict.com/2019/12/31/year-i...,NA,2019


In [8]:
df.to_csv("df.csv", encoding='utf-8', index=False)